In [4]:
'''
sensor_surfaceに関する処理
参考
https://qiita.com/mhangyo/items/76db7c6a6ebba6cf4330

dtypeに関する資料
http://discexuno.wp.xdomain.jp/machine-learning/python_fromfile_fromstring/
https://docs.python.org/3/library/struct.html

3Dfigure
https://morvanzhou.github.io/tutorials/data-manipulation/plt/3-5-3d/
'''
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import cv2 
from scipy.fftpack import fft,ifft
import os
import glob
from pylab import *
import math
import matplotlib.animation as animation


In [ ]:
'''
付録ーーデータの展開法

一つの実験についてのデータは四つのフォルダーに分けております。
そのうち、「Height」と書いてるフォルダーに変異データが入ってます。
そのフォルダーの中には縦方向（Y軸）の画素分のデータファイルがあります。
一つのファイルには一行についての画像枚分のデータが横方向（X軸）の画素分重ねております。

つまり、X行、Y棚、Fフレームのデータポイントを見つけるには
Y個目のファイルを開いて、X×フレーム数（16666frames=500fps×33.3s）＋F　のインデックスでXYFのデータがあります。

データ自体は64bit doubleです。バイトオーダーはビッグエンディアンです (byte order: big endian)。ヘッダーはないです。

どうぞよろしくお願いします。
ライオン

他の設定に関する情報元(resolution/framerate/downsampling,etc)　　\\FS1\Maedalab\home\junjie-hua\wave\fourD_sensor\rawdata\20191101\Captures
'''

#f = open(r"\\FS1\Maedalab\home\junjie-hua\wave\fourD_sensor\rawdata\20191101\test1_belly\Height\Height Row 0000.data",mode='rb')#0列目
#topo = np.fromfile(f, dtype='float64',sep='').reshape(256,20000) #float64の形で読み込むと，5120000の数列ができました．bellyの計測時間=20*1000msため，256*20000とreshapeしました．x軸上のサンプリング点が256個となります．



'\n付録ーーデータの展開法\n\n一つの実験についてのデータは四つのフォルダーに分けております。\nそのうち、「Height」と書いてるフォルダーに変異データが入ってます。\nそのフォルダーの中には縦方向（Y軸）の画素分のデータファイルがあります。\n一つのファイルには一行についての画像枚分のデータが横方向（X軸）の画素分重ねております。\n\nつまり、X行、Y棚、Fフレームのデータポイントを見つけるには\nY個目のファイルを開いて、X×フレーム数（16666frames=500fps×33.3s）＋F\u3000のインデックスでXYFのデータがあります。\n\nデータ自体は64bit doubleです。バイトオーダーはビッグエンディアンです (byte order: big endian)。ヘッダーはないです。\n\nどうぞよろしくお願いします。\nライオン\n\n他の設定に関する情報元(resolution/framerate/downsampling,etc)\u3000\u3000\\FS1\\Maedalab\\home\\junjie-hua\\wave\x0courD_sensor\rawdata\x8191101\\Captures\n'

In [ ]:
path = "//FS1//Maedalab//home//junjie-hua//wave//fourD_sensor//rawdata//20191101//test2_display_surface//Height" #belly上の計測データ元　
files = os.listdir(path) #all files in the faulder　縦方向(y軸)のデータファイル
s = []
n = 0

os.chdir("//FS1//Maedalab//home//junjie-hua//wave//fourD_sensor//rawdata//20191101//test3_sensor_surface//Height") #local()関数を利用するためのコードです
for file in glob.glob("*.data"): 
    f = open(path + "//" + file ,mode='rb')#n列目　rb= read binary
    name = 'topo'+str(n)  #一つのファイルには一行についての画像枚分のデータが横方向（X軸）の画素分のデータを変数topo+str(n)に入れます
    locals()['topo'+str(n)]= np.fromfile(f, dtype='>d').reshape(256,33332) 
    # https://docs.python.org/3/library/struct.html
    # ">" means big endian, "d" means double. 8532992の数列ができました．bellyと同じレゾルーションで計測したため，x軸上のサンプリング点が256個となります．
    print(file,"has ended")
    n = n + 1 
     
print(n) #y軸上サンプリング点の数を数えます--252個(なぜ256ではない？)

Height Row 0000.data has ended
Height Row 0001.data has ended
Height Row 0002.data has ended
Height Row 0003.data has ended
Height Row 0004.data has ended
Height Row 0005.data has ended
Height Row 0006.data has ended
Height Row 0007.data has ended
Height Row 0008.data has ended
Height Row 0009.data has ended
Height Row 0010.data has ended
Height Row 0011.data has ended
Height Row 0012.data has ended
Height Row 0013.data has ended
Height Row 0014.data has ended
Height Row 0015.data has ended
Height Row 0016.data has ended
Height Row 0017.data has ended
Height Row 0018.data has ended
Height Row 0019.data has ended
Height Row 0020.data has ended
Height Row 0021.data has ended
Height Row 0022.data has ended
Height Row 0023.data has ended
Height Row 0024.data has ended
Height Row 0025.data has ended
Height Row 0026.data has ended
Height Row 0027.data has ended
Height Row 0028.data has ended
Height Row 0029.data has ended
Height Row 0030.data has ended
Height Row 0031.data has ended
Height R

In [ ]:
'''
color mapのアニメーション
'''

# (x,y,t)のデータを取り出すための関数 
names = locals()
def f(x,y,t): 
    k = names['topo'+str(y)][x,t]  #topo_str(y)の数列から，(x,t)のデータを取り出します
    return k
#print(f(255,251,19999))


# X, Y value
x = np.linspace(0,256-1,256)
y = np.linspace(0,252-1,252)
X,Y = np.meshgrid(x,y)  #サンプリング点の数：x軸256個， y軸252個


Z = [[0 for i in range(X.shape[1])] for i in range(Y.shape[0])]

print("start")


for t in range(20000):
    if t % 100 == 0:
        for i in range(X.shape[0]):
            for j in range(X.shape[1]):
                Z[i][j] = f(j,i,t)     
        Z_array = np.array(Z)# turn list into array
        
       # color mapを作成する
        plt.pcolormesh(X, Y, Z_array, cmap='hsv') # 等高線図の生成。cmapで色付けの規則を指定する。
        #plt.pcolor(X, Y, Z, cmap='hsv') # 等高線図の生成。cmapで色付けの規則を指定する。

        pp=plt.colorbar (orientation="vertical") # カラーバーの表示 
        pp.set_label("Label", fontname="Arial", fontsize=24) #カラーバーのラベル

        plt.xlabel('X', fontsize=24)
        plt.ylabel('Y', fontsize=24)
        plt.title("T = %s ms"% (t))

        print(t)
        plt.savefig("//FS1//Maedalab//home//junjie-hua//wave//fourD_sensor//rawdata//20191101//Captures//animation//color_map//test3_sensor_surface//" + str(t) + ".png")
        plt. close()
    else:
        continue
